# Interactor Finder
The `InteractorFinder` class is used to identify specifically linked nodes to your starting case. Here, we show how one initializes this class by choosing the MAPT protein with phosphorylation modification, and finds all causally linked neighbors.

In [ ]:
#!pip install drugintfinder
from drugintfinder.finder import InteractorFinder

import pandas as pd

# Initialize with base information
finder = InteractorFinder(
    node_name="MAPT",  # Target node should have the MAPT gene symbol
    node_type="protein",  # Target node should be a protein
    pmods=["pho"],  # Target node should have a phosphorylation modification
    neighbor_edge_type="causal",  # The direct interactors should be related using causal edge types
    print_sql=False,  # Print the query as it's presented to OrientDB server
    neighbor_edge_filters=["tau=true"],  # Additional interactor edge filters
)

# Select for matching starting protein nodes (i.e. MAPT protein) and find all interactors
neighbors = finder.find_interactors()
neighbors

### Druggable Interactors
While knowing the neighbors of select nodes is useful, knowing which ones can be targeted by drugs and compounds is even more informative. The InteractorFinder class has a method for searching out those special neighbors. By default, these neighbors are isolated to proteins as the KG restricts drug-target interactions to those occurring between proteins and compounds.

In [ ]:
druggable_ints = finder.druggable_interactors()
druggable_ints

# Ranker
The `Ranker` class is used to generate useful statistics about identified druggable interactors. Because this scoring is restricted to druggable interactors, only the starting node and pmods are needed to initialize.  

The ranking algorithm requires a bit of information to score everything, the first time it is used, it will need to download information on BioAssays and other resources which it will then store in a locally made SQLite database. The total space used is less than 100 MB.  

The download will take a couple of minutes during the first ranking, but subsequent rankings will be much faster.

In [ ]:
from drugintfinder.ranker import Ranker

ranker = Ranker(symbol="MAPT", pmods=["pho"], reward=1, penalty=-1)
ranker.rank()  # Performs the ranking
summary = ranker.summarize()  # Create a summary os the statistics
summary

In [ ]:
bioassays_per_drug = []

for interactor in summary.Target.unique():
    drugs = len(summary[summary.Target == interactor].Drug.unique())
    bioassays = summary[summary.Target == interactor]["Number of BioAssays for Target"].iloc[0]
    bioassays_per_drug.append({"Protein": interactor, "BioAssays per Drug": bioassays // drugs})

ratio_df = pd.DataFrame(bioassays_per_drug)
ratio_df.sort_values("BioAssays per Drug", ascending=False)[:10]

In [ ]:
import pandas as pd
summary[summary.Target == "APP"]["Number of BioAssays for Target"].iloc[0]

In [ ]:
summary.sort_values(by="Number of BioAssays for Target", ascending=False)

# PPI Analysis
Next, we perform an analysis of the identified proteins using information gathered by [e(BE:L)](https://github.com/e-bel/ebel).
The following commands will download data from 4 major PPI databases: BioGRID, Pathway Commons, StringDB, and IntAct,
and check which pathways/interactions are known for every identified secondary target.

In [ ]:
# Uncomment the following line if you need to install e(BE:L)
#!pip install ebel git+https://github.com/orientechnologies/pyorient

import pandas as pd
from ebel import Bel
bel = Bel()

## Download PPI Information
The following cell downloads information from the PPI databases and inserts into a RDBMS (SQLlite [default] or MySQL).  
**WARNING** This step may take awhile.

In [ ]:
bel.biogrid.update()
bel.intact.update()
bel.stringdb.update()
bel.pathway_commons.update()

## Gather Hits
Now we check each PPI database for associated information on each secondary target.

In [ ]:
proteins = list(summary.Target.unique())

### Pathway Commons

In [ ]:
sql = f"""Select
    pc.participant_a a,
    pc.interaction_type int_type,
    pc.participant_b b,
    group_concat(distinct pn.name) pathway_names,
    group_concat(distinct s.source) sources,
    group_concat(distinct p.pmid) pmids
from
    pathway_commons pc left join
    pathway_commons__pathway_name pc_pn on (pc.id=pc_pn.pathway_commons_id) left join
    pathway_commons_pathway_name pn on (pc_pn.pathway_commons_pathway_name_id = pn.id) left join
    pathway_commons__source pc_s on (pc.id=pc_s.pathway_commons_id) left join
    pathway_commons_source s on (pc_s.pathway_commons_source_id=s.id) left join
    pathway_commons_pmid p on (p.pathway_commons_id=pc.id)
where
    (pc.participant_a in {proteins} and pc.participant_b = 'MAPT') or
    (pc.participant_b in {proteins} and pc.participant_a = 'MAPT')
group by
    pc.participant_a, pc.interaction_type, pc.participant_b"""

pc_hits = pd.read_sql(sql, engine)

### BioGRID

In [ ]:
sql = f"""Select
    ia.symbol a,
    ib.symbol b,
    bes.experimental_system,
    bes.experimental_system_type
from
    biogrid b inner join
    biogrid_interactor ia on (b.biogrid_a_id=ia.biogrid_id) inner join
    biogrid_interactor ib on (b.biogrid_b_id=ib.biogrid_id) inner join
    biogrid_experimental_system bes on (b.experimental_system_id=bes.id)
where
    (ia.symbol = 'MAPT' and ib.symbol in {proteins}) or
    (ib.symbol = 'MAPT' and ia.symbol in {proteins})"""

biogrid_hits = pd.read_sql(sql, engine)

### IntAct

In [ ]:
sql = f"""Select
    ha.symbol as symbol_a,
    hb.symbol as symbol_b,
    i.confidence_value, 
    i.detection_method, 
    i.interaction_type, 
    i.pmid
from 
    intact i inner join 
    hgnc_uniprot hua on (i.int_a_uniprot_id=hua.accession) inner join 
    hgnc ha on (hua.hgnc_id=ha.id) inner join 
    hgnc_uniprot hub on (i.int_b_uniprot_id=hub.accession) inner join 
    hgnc hb on (hub.hgnc_id=hb.id)
where 
    (ha.symbol='MAPT' and hb.symbol in {proteins}) or
    (hb.symbol='MAPT' and ha.symbol in {proteins})
order by confidence_value desc
"""
intact_hits = pd.read_sql(sql, engine)

### StringDB

In [ ]:
sql = f"""Select * 
from 
    stringdb 
where 
    (symbol1='MAPT' and symbol2 in {proteins}) or
    (symbol2='MAPT' and symbol1 in {proteins})
order by combined_score desc
"""
stringdb_hits = pd.read_sql(sql, engine)

# Connecting to a Different Knowledge Graph
By default, this package connects to the Alzheimer's Disease based Knowledge Graph (KG) developed under the MAVO project, available at https://graphstore.scai.fraunhofer.de. There are other KGs available, however, and here you can choose to connect to a different one if desired.

The commented out code shows how one can connect instead to the COVID KG.

In [ ]:
from ebel_rest import connect
connect(user="covid_user", password="covid", db_name="covid", server="https://graphstore.scai.fraunhofer.de")